<a href="https://colab.research.google.com/github/Mutum/Mutum-computer_vision_EVA/blob/main/Session%202.5/Session_2_5_PyTorch_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


In [9]:
train_set = torchvision.datasets.MNIST(
    root="MNIST_data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [10]:
class custom_data():

    def __init__(self,batch_size) -> None:
        self.train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
        self.a_num_numpy = None
        # self.a_num_torch = None
        self.genrate_rand() 
     
    def genrate_rand(self):
        
        a_num_numpy = np.random.choice(range(0,10,1))
        # a_num_numpy = a_num_numpy.astype('float')
        a_num_numpy_oneHot = np.array([1 if num == a_num_numpy else 0 for num in range(0,10) ])
        a_num_torch = torch.from_numpy(a_num_numpy_oneHot)
        
        self.a_num_numpy = a_num_numpy
        # self.a_num_torch = a_num_torch
        


In [11]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)
    self.out_sum = nn.Linear(in_features=60, out_features=18)
    
    # self.fc_rand = nn.Linear(in_features=192, out_features=192)

  def forward(self, t, a_num_numpy):
    # input layer
    # here are the batch of images 
    x = t 
    # conv1 layer
    x = self.conv1(x) 
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) 
    # conv2 layer
    x = self.conv2(x) 
    x = F.relu(x) 
    x = F.max_pool2d(x, kernel_size=2, stride=2) 
    # reshapre
    x = x.reshape(-1, 12 * 4 * 4)

    # the random number from 0-9 in numpy
    x_rand = a_num_numpy

    # add the random number to our previous image convolution layer
    x_new = torch.add(x, x_rand)
    
    # fc1 layer
    x_new = self.fc1(x_new)
    x_new = F.relu(x_new)
    
    # fc2 layer
    x_new = self.fc2(x_new)
    x_new_ = F.relu(x_new)
    
    # output prediction for the 
    x_new = self.out(x_new_)

    #  add for another output of the total_sum prediction
    x_sum = self.out_sum(x_new_)
    
    return x_new, x_sum


In [12]:
def get_num_correct(preds, labels):
  """ return accurary """
  return preds.argmax(dim=1).eq(labels).sum().item()


In [13]:
# initiating our data input 

# custom data class initiated
training_data = custom_data(batch_size= 64)

# images and labels
train_loader = training_data.train_loader

# the random number generated
a_num_numpy = training_data.a_num_numpy 


In [14]:
# Model training

network = Network()
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(20):

    total_loss = 0
    total_correct_image = 0
    total_correct_sum_output = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 
        sum_labels = labels + a_num_numpy

        preds, pred_sum = network(images, a_num_numpy) # Pass Batch
        # print(preds.shape)
        # loss = F.cross_entropy(preds, labels)
        # pred_sum = pred_sum + x_rand
        loss_1 = F.cross_entropy(preds, labels) # Calculate Loss
        loss_2 = F.cross_entropy(pred_sum, sum_labels)

        loss = loss_1 + loss_2
        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct_image  += get_num_correct(preds, labels)
        total_correct_sum_output += get_num_correct(pred_sum, sum_labels)

    print(
        "epoch:", epoch, 
        " total_correct_image:", total_correct_image , 
        " total_correct_sum_output:", total_correct_sum_output, 
        " loss:", total_loss
    )

epoch: 0  total_correct_image: 6356  total_correct_sum_output: 6313  loss: 4333.2324986457825
epoch: 1  total_correct_image: 6496  total_correct_sum_output: 6465  loss: 4321.711705684662
epoch: 2  total_correct_image: 6589  total_correct_sum_output: 6560  loss: 4320.180404663086
epoch: 3  total_correct_image: 6640  total_correct_sum_output: 6658  loss: 4319.501060009003
epoch: 4  total_correct_image: 6673  total_correct_sum_output: 6670  loss: 4319.139639377594
epoch: 5  total_correct_image: 6682  total_correct_sum_output: 6675  loss: 4318.807169914246
epoch: 6  total_correct_image: 6680  total_correct_sum_output: 6680  loss: 4318.538893699646
epoch: 7  total_correct_image: 6694  total_correct_sum_output: 6694  loss: 4318.357453346252
epoch: 8  total_correct_image: 6691  total_correct_sum_output: 6691  loss: 4318.272194862366
epoch: 9  total_correct_image: 6691  total_correct_sum_output: 6691  loss: 4318.239808082581
epoch: 10  total_correct_image: 6691  total_correct_sum_output: 6691 